In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%ls


drive/  sample_data/


In [ ]:
%ls 

drive/  sample_data/


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.

class MvsNM():
    IMG_SIZE = 50
    mask = "/content/drive/My Drive/ds/dataset/with_mask"
    nomask = "/content/drive/My Drive/ds/dataset/without_mask"
    LABELS = {mask: 0, nomask: 1}
    training_data = []

    mask_count = 0
    nomask_count = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  
                        if label == self.mask:
                            self.mask_count += 1
                        elif label == self.nomask:
                            self.nomask_count += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Mask:',mvsnm.mask_count)
        print('No mask:',mvsnm.nomask_count)

if REBUILD_DATA:
    mvsnm = MvsNM()
    mvsnm.make_training_data()

training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

import torch

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)

if torch.cuda.is_available():
    device = torch.device("cuda:0")  
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

net = Net().to(device)

print(net)

import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]
print(len(train_X), len(test_X))

/content/drive/My Drive/ds/dataset/with_mask


  0%|          | 0/1948 [00:00<?, ?it/s]

/content/drive/My Drive/ds/dataset/without_mask


100%|██████████| 1948/1948 [12:49<00:00,  2.53it/s]


Mask: 2474
No mask: 1948
4422
Running on the GPU
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)
442
3980 442


In [ ]:
import time
BATCH_SIZE = 16
EPOCHS = 30
start_time=time.time()
def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    for epoch in range(EPOCHS):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()

            optimizer.zero_grad()   # zero the gradient buffers
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")

train(net)
print("--- %s seconds ---" % (time.time() - start_time))

test_X.to(device)
test_y.to(device)

def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

test(net)
correct = 0
total = 0
for i in tqdm(range(0, len(test_X), BATCH_SIZE)):

    batch_X = test_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50).to(device)
    batch_y = test_y[i:i+BATCH_SIZE].to(device)
    batch_out = net(batch_X)

    out_maxes = [torch.argmax(i) for i in batch_out]
    target_maxes = [torch.argmax(i) for i in batch_y]
    for i,j in zip(out_maxes, target_maxes):
        if i == j:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

Epoch: 0. Loss: 0.14336919784545898
Epoch: 1. Loss: 0.10330641269683838
Epoch: 2. Loss: 0.02647865191102028
Epoch: 3. Loss: 0.020516395568847656
Epoch: 4. Loss: 0.012019835412502289
Epoch: 5. Loss: 0.00792643427848816
Epoch: 6. Loss: 0.0009657948976382613
Epoch: 7. Loss: 0.016265179961919785
Epoch: 8. Loss: 0.00012442746083252132
Epoch: 9. Loss: 3.4991669963346794e-05
Epoch: 10. Loss: 0.0001611625193618238
Epoch: 11. Loss: 0.00025839515728875995
Epoch: 12. Loss: 1.2523730219982099e-05
Epoch: 13. Loss: 8.403477113461122e-05
Epoch: 14. Loss: 1.8811500694937422e-06
Epoch: 15. Loss: 6.134577574812283e-07
Epoch: 16. Loss: 9.656370821176097e-06
Epoch: 17. Loss: 0.0012540564639493823
Epoch: 18. Loss: 6.022842171660159e-06
Epoch: 19. Loss: 0.003990841563791037
Epoch: 20. Loss: 0.0005229134112596512
Epoch: 21. Loss: 8.151933172939607e-08
Epoch: 22. Loss: 9.99870849227591e-07
Epoch: 23. Loss: 1.0781891823796741e-08
Epoch: 24. Loss: 3.4340663646048597e-09
Epoch: 25. Loss: 9.808611594053218e-07
Ep

 20%|█▉        | 87/442 [00:00<00:00, 867.72it/s]

Epoch: 29. Loss: 8.833613840408816e-10
--- 36.85317254066467 seconds ---


100%|██████████| 28/28 [00:00<00:00, 349.66it/s]

Accuracy:  0.921
Accuracy:  0.921
